# Scikit-Learn cross_validate

Prin metoda corss_validate putem să ne uităm peste mai mulți metrici de performanță a modelului și de asemenea putem să vedem și cât timp a luat partea de antrenare și validare a modelului pentru fiecare validare în parte.

In [1]:
# importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# read the data into a DataFrame
df = pd.read_csv('../data/DATA/Advertising.csv')

In [3]:
# print the head of the data
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [4]:
# importing the functions from sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

# split the data into Features and labels
X = df.drop('sales', axis=1)
y = df['sales']

# split the data into train-test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# create a scaler, fit the scaler and transform the data
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


In [5]:
# create a model
model = Ridge(alpha=100)

După ce am preprocesat datele și am creat un model de Ridge Regression o să importăm metoda de cross_validate. Metoda se găsește în modulul de model_selection din sklearn

In [6]:
from sklearn.model_selection import cross_validate

Ce este diferit de metoda cross_val_score este faptul că pentru parametrul de scoring acum putem să oferim ca și valoare o listă de string-uri

In [9]:
scores = cross_validate(model, X_train, y_train, scoring=['neg_mean_squared_error', 'neg_mean_absolute_error'], cv=10)

In [10]:
scores

{'fit_time': array([0.00281811, 0.00163102, 0.00201607, 0.00159693, 0.00249696,
        0.0014751 , 0.00151873, 0.00135612, 0.00127721, 0.00170493]),
 'score_time': array([0.00212193, 0.00074697, 0.00254011, 0.00078583, 0.00117302,
        0.00118208, 0.00101829, 0.00098777, 0.00092173, 0.00119495]),
 'test_neg_mean_squared_error': array([ -6.06067062, -10.62703078,  -3.99342608,  -5.00949402,
         -9.14179955, -13.08625636,  -3.83940454,  -9.05878567,
         -9.05545685,  -5.77888211]),
 'test_neg_mean_absolute_error': array([-1.8102116 , -2.54195751, -1.46959386, -1.86276886, -2.52069737,
        -2.45999491, -1.45197069, -2.37739501, -2.44334397, -1.89979708])}

Ceea ce se retunrează este un dicționar de valori unde pe lângă cele două metrici ('neg_mean_squared_error', 'neg_mean_absolute_error') mai avem și date despre fitting și score_time, unde fitting reprezintă cât de mult a durat partea de antrenare a modelului și score_time face referire la timpul pentru calcularea erorii. Din moment ce este un dicționar, putem să îl transformăm ușor într-un DataFrame în pandas

In [11]:
pd.DataFrame(scores)

,fit_time,score_time,test_neg_mean_squared_error,test_neg_mean_absolute_error
0,0.002818,0.002122,-6.060671,-1.810212
1,0.001631,0.000747,-10.627031,-2.541958
2,0.002016,0.002540,-3.993426,-1.469594
3,0.001597,0.000786,-5.009494,-1.862769
4,0.002497,0.001173,-9.141800,-2.520697
5,0.001475,0.001182,-13.086256,-2.459995
6,0.001519,0.001018,-3.839405,-1.451971
7,0.001356,0.000988,-9.058786,-2.377395
8,0.001277,0.000922,-9.055457,-2.443344
9,0.001705,0.001195,-5.778882,-1.899797


În acest fel avem toate datele prezente într-un DataFrame, iar de aici putem să extragem valorile medii sau să facem partea de vizualizare a datelor.

In [12]:
scores_df = pd.DataFrame(scores)

In [15]:
abs(scores_df.mean())

fit_time                        0.001789
score_time                      0.001267
test_neg_mean_squared_error     7.565121
test_neg_mean_absolute_error    2.083773
dtype: float64

 În continuare, dacă nu suntem mulțumiți cu caliatea modelului putem să creem un nou model cu alte valori pentru hyperparametrii și să rerulăm partea de corss_validate

In [16]:
model2 = Ridge(alpha=1)

scores = cross_validate(model2, X_train, y_train, cv=10, scoring=['neg_mean_squared_error', 'neg_mean_absolute_error'])
scores_df = pd.DataFrame(scores)
abs(scores_df.mean())

fit_time                        0.001169
score_time                      0.000783
test_neg_mean_squared_error     3.323018
test_neg_mean_absolute_error    1.308467
dtype: float64

Din nou, la final trebuie să antrenăm modelul pe train-set și să facem predicții pe setul de date de testare

In [18]:
model2.fit(X_train, y_train)
y_pred = model2.predict(X_test)

In [19]:
from sklearn.metrics import mean_squared_error

RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
RMSE

1.5228334050147283

## Recapitulare

În această parte am învățat următoarele:

    1. Diferența dintre cross_val_score și cross_validate este faptul că pentru cross_validate la parametrul de scoring putem să oferim o listă de valori, de metrici care să fie returnați

        scores = cross_val_score(model2, X_train, y_train, cv=10, scoring='neg_mean_squared_error')

        scores = cross_validate(model2, X_train, y_train, cv=10, scoring=['neg_mean_squared_error', 'neg_mean_absolute_error'])

    2. cross_validate mai retunrează și date despre fit_time și score_time care reprezintă timpul necesar pentru antrenare și validare pentru fiecare fold în parte

    3. cross_validate returnează un dicționar de elemente pe când cross_cal_score returnează un array de valori